# XHARPy Example: CpNa with GPAW
 - Program: GPAW
 - RPBE calculation
 - Constant core calculation
 - Read xd.hkl
 - Disorder on special position / Skip atoms
 

This is an XHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up. Specifically it is a demonstration of skipping atoms, which are disordered on a special position can be skipped from the calculation with skip_symm in the computation_dict. In this case we are fortunate enough, that this is all it takes to lead the refinement to a reasonable result. Otherwise restraints would be needed, which is something, that is not implemented at the moment

In [ ]:
from xharpy import (xd_hkl2pd, cif2data, lst2constraint_dict, create_construction_instructions,
                    refine, write_cif, write_res, write_fcf, add_density_entries_from_fcf)
import os

In [ ]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
filename = 'xharpy'

### Loading Data

Notice how the constraints for H1 are copied from C1. This means we do not need an anisotropic H refinement in shelxl

In [ ]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [ ]:
hkl = xd_hkl2pd('xd.hkl')

In [ ]:
constraint_dict = lst2constraint_dict(
    'iam.lst'
)

constraint_dict['H1'] = constraint_dict['C1']

### Setting options
The central option for the special position is skip_symm. Unfortunately skip_symm only operates with indexes, as the inner routine does not know how the atoms are named.

The first index, that is used as key is the atom_index. You can get it from the atom_table. The list afterwards are symmetry indexes which are skipped. These correspond to the indexes in both the symm_strings or the symm_mats_vecs

In [ ]:
refinement_dict = {
    'f0j_source': 'gpaw_mpi',
    'reload_step': 1,
    'core': 'constant',
}

In [ ]:
computation_dict = {
    'xc': 'RPBE',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.175,
    'gridinterpolation': 4,
    'symm_equiv': 'once',
    'convergence':{'density': 1e-7},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2,
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw'),
    'skip_symm': {
        2: [2, 3, 6, 7], # For atom 2: Skip symmetry elements 2, 3, 6 and 7
        3: [2, 3, 6, 7],
        4: [2, 3, 6, 7]
    }
}

### Refine

In [ ]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [ ]:

parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

### Write Data to disk

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_res(
    out_res_path=os.path.join(output_folder, filename + '_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [ ]:
write_cif(
    output_cif_path=os.path.join(output_folder, filename + '.cif'),
    cif_dataset=filename,
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [ ]:
add_density_entries_from_fcf(
    os.path.join(output_folder, 'xharpy.cif'),
    os.path.join(output_folder, 'xharpy_6.fcf')
)